In [ ]:
# %matplotlib inline
from matplotlib import pyplot as plt
import os
import cv2
import requests
from os import path
import numpy as np
from tqdm import tqdm
from PIL import Image
import pickle
from rectangle import Rectangle
import helpers
from helpers import *
import time
import pickle

kindle_ar = (1430,1080)

if not path.exists(f'dump'):
    os.mkdir('dump')

manga_name = 'Akira'

In [ ]:
os.environ["PATH"] += ":"+os.getcwd()
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service

# Set up Firefox options
firefox_options = Options()s
firefox_options.add_argument('--headless')  # Run Firefox in headless mode
firefox_options.binary_location = '/usr/bin/firefox'  # Path to Firefox binary

# Set up the Firefox WebDriver
service = Service('./geckodriver')
driver = webdriver.Firefox(service=service, options=firefox_options)

In [ ]:
driver.get("https://mangasee123.com/manga/"+manga_name)
# with open('mangasee.html', 'w', encoding='utf-8') as f:
#     f.write(driver.page_source)
# selector = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.ShowAllChapters')))
# selector.click()

chapter_links = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a[href*="read-online"]')]

chapter_links = reversed(chapter_links)
pages = {}
for chapter in chapter_links:
    driver.get(chapter)

    selector = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'i.fa-arrows-alt-v')))
    selector.click()

    img_links = [img.get_attribute('src') for img in driver.find_elements(By.CSS_SELECTOR, 'img.img-fluid')]
    pages[chapter.split('/')[-1]] = img_links
with open(manga_name+'.pkl', 'wb') as f:
    pickle.dump(pages, f)

In [ ]:
def process_page(url, draw_panel=False):
    try:
        raw_img = requests.get(url, stream=True).raw
        img = np.array(Image.open(raw_img))
        if img.ndim==2:
            img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
    except:
        return

    # if not path.exists(f'dump/{manga_name}/{chapter_counter:04d}'):
    #     os.mkdir(f'dump/{manga_name}/{chapter_counter:04d}')
    # img = cv2.imread('0012-002.png')
    # plot_crops(img, [])
    contours = get_contours(img)
    contours = remove_specks_and_lines(img, contours)
    crops = get_crops(img, contours)
    crops = merge_overlapping_crops(crops)
    crops = remove_small_crops(img, crops)
    helpers.debug = False

    text_crops = get_text_crops(img)
    text_crops = remove_small_crops(img, text_crops, factor=0.5)

    stats = compute_statistics(img, crops)
    crops.extend(text_crops)
    crops = merge_overlapping_crops(crops)

    # debug = True
    # plot_frames(img, crops, True)
    edge_graph = build_crops_edge_graph(crops, img.shape)
    # print(*[(i,j) for i,j in enumerate(edge_graph)], sep='\n')
    sorted_crops = sort_edge_graph(crops, edge_graph)
    idx = 0
    if not sorted_crops:
        sorted_crops = [Rectangle(0,0,img.shape[1],img.shape[0])]
    if stats['ignored']>0.20 and stats['overlapped']>0.01: 
        sorted_crops = [Rectangle(0,0,img.shape[1],img.shape[0])]
    if len(sorted_crops)==1:
        sorted_crops = [Rectangle(0,0,img.shape[1],img.shape[0])]

    image = plot_frames(img, sorted_crops, True, show=helpers.debug)

    to_write = []
    for c in sorted_crops:
        gray_crop = cv2.cvtColor(img[c.y:c.z, c.x:c.w], cv2.COLOR_BGR2GRAY)

        if gray_crop.shape[1]/gray_crop.shape[0]>0.80 or c.area < img.shape[0]*img.shape[1]*0.5:
            to_write.append(gray_crop)
        elif len(sorted_crops)==1:
            text_crops = merge_vertical_overlapping_crops(text_crops)
            if len(text_crops)==0:
                middleh = c.height//2+c.y
            elif len(text_crops)==1:
                middleh = c.height//2+c.y
                if text_crops[0].y<=middleh and middleh<=text_crops[0].z:
                    middleh = text_crops[0].y if text_crops[0].y-middleh < middleh-text_crops[0].z else text_crops[0].z
            else: # find the widest gap
                def pairwise(iterable):
                    a = iter(iterable)
                    return zip(a, a)
                widest_gap = max([(idx,j.y-i.z) for idx,(i,j) in enumerate(list(pairwise(text_crops)))], key=lambda t: t[1])
                middleh = text_crops[widest_gap[0]].z+widest_gap[1]//2
            cv2.line(image, (c.x, middleh), (c.w, middleh), (0, 255, 0), thickness=3)
            to_write.extend([h for h in np.split(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), [middleh])])
        else:
            to_write.append(gray_crop)

    if draw_panel:
        return image, stats

    if helpers.debug:
        plot_crops(image,[])
    return [resizeAndPad(np.rot90(gc), kindle_ar, 255) for gc in to_write]


In [ ]:
# DEBUG CELL
if not path.exists(f'dump/{manga_name}'):
    os.mkdir(f'dump/{manga_name}')

    pages = pickle.load(f)
    
helpers.debug = False

image,stats = process_page('https://official-ongoing-2.gamindustri.us/manga/Onepunch-Man/0037-016.png', draw_panel=False)
plt.figure(figsize=(16,16))
plt.imshow(image)
plt.show()

In [ ]:
# DEBUG CELL
with open(manga_name+'.pkl', 'rb') as f:
    pages = pickle.load(f)
# print(pages[0])
for urls in pages.values():
    for url in urls:
        image,stats = process_page(url, draw_panel=True)
        plt.figure(figsize=(16,16))
        plt.imshow(image)
        plt.show()
        print(stats)

In [ ]:
import multiprocessing

if not path.exists(f'dump/{manga_name}'):
    os.mkdir(f'dump/{manga_name}')

with open(manga_name+'.pkl', 'rb') as f:
    pages = pickle.load(f)

def update_result(result):
    image, stats = result

def write_result(pages, chapter_counter, page_counter):
    idx = 0
    for page in pages:
        cv2.imwrite(f"dump/{manga_name}/{chapter_counter:04d}/{page_counter:03d}-{idx:03d}.jpg", page, [int(cv2.IMWRITE_JPEG_QUALITY), 20])
        idx+=1
        

def call_and_save(args):
    url, chapter_counter, page_counter = args
    try:
        write_result(process_page(url), chapter_counter, page_counter)
    except Exception as e:
        print(url, chapter_counter, page_counter, e)

def chap_generator():
    for chapter_counter,chap in enumerate([*pages.keys()]):
        # if chapter_counter!=3: continue
        os.system(f'mkdir -p dump/{manga_name}/{chapter_counter:04d}')
        page0 = np.zeros([*reversed(kindle_ar)])
        cv2.putText(img=page0, text=f'Chapter {chapter_counter:4d}', org=(150, 250), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=3, color=(255),thickness=3)
        cv2.putText(img=page0, text=f'{chap}', org=(150, 350), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255),thickness=3)
        page0 = np.rot90(page0)
        page_counter = 0
        write_result([page0], chapter_counter, page_counter)
        for url in pages[chap]:
            page_counter+=1
            yield url, chapter_counter, page_counter

pool = multiprocessing.Pool(64)
for i in tqdm(pool.imap_unordered(call_and_save, chap_generator()), total=sum([len(i) for i in pages.values()])): pass


In [ ]:
# find /nfs/iir/home/lpuglia/repository/kumiko -name .ipynb_checkpoints -exec rm -rf {} \;
# export PATH=$PATH:/nfs/iir/home/lpuglia/repository/kumiko/dump
# /local_storage0/lpuglia/anaconda3/envs/kumiko/bin/kcc-c2e -p K34 dump/{manga_name}

# os.system(f"cd dump; zip -FSr {manga_name} {manga_name} -x '*/\.*'")